In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
from dataclasses import dataclass, field
import numpy as np, pandas as pd, matplotlib.pyplot as plt
!pip install networkx -q
import networkx as nx
from matplotlib.colors import ListedColormap
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150,
                     'axes.titlesize': 'large', 'axes.labelsize': 'medium',
                     'xtick.labelsize': 'small', 'ytick.labelsize': 'small'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for Advanced Agent-Based Modeling (ABM).")

# Chapter 10.1: Agent-Based Models: A "Bottom-Up" Approach to Economic Complexity

---

### Table of Contents
1.  [**The Philosophy and Building Blocks of ABM**](#philosophy)
    - [Emergence, Heterogeneity, and Bounded Rationality](#concepts)
    - [The ODD Protocol for Describing ABMs](#odd)
2.  [**Foundational Model 1: Schelling's Segregation Model**](#schelling)
    - [Emergence and Tipping Points](#schelling-analysis)
3.  [**Foundational Model 2: Axelrod's Tournament and the Evolution of Cooperation**](#axelrod)
    - [Implementing the Iterated Prisoner's Dilemma](#axelrod-impl)
4.  [**Application 1: Financial Market ABMs**](#finance-abm)
    - [The Kirman (1993) "Ants" Model of Herd Behavior](#kirman)
    - [An Artificial Stock Market (Lux & Marchesi)](#asm)
5.  [**Application 2: A Simple Agent-Based Macroeconomic Model**](#macro-abm)
    - [Generating Endogenous Business Cycles](#macro-impl)
6.  [**Strengths, Weaknesses, and Validation**](#validation)
7.  [**Summary and Key Takeaways**](#summary)
8.  [**Exercises**](#exercises)

<a id='philosophy'></a>
## 1. The Philosophy and Building Blocks of ABM

The macroeconomic models explored so far are masterpieces of the **"top-down"** or **deductive** approach. They typically feature a high degree of aggregation—often collapsing the behavior of millions of diverse individuals into a single, hyper-rational **representative agent**—to solve for the unique, stable **equilibrium** of the system.

This methodology is powerful, but it is less suited to explaining phenomena rooted in **heterogeneity**, **complex network interactions**, and **out-of-equilibrium dynamics**. How do financial crises cascade through the system? How do market bubbles form and burst? How do patterns of extreme wealth inequality arise?

**Agent-Based Modeling (ABM)** offers a radically different, **"bottom-up"** or **generative** philosophy. Instead of assuming an aggregate equilibrium, an ABM seeks to *generate* aggregate patterns from the ground up, a concept often framed by Joshua Epstein's question: "Can you grow it?"

<a id='concepts'></a>
### Emergence, Heterogeneity, and Bounded Rationality

The core tenets of the ABM approach are:
1.  **Heterogeneity:** Agents are not identical. They can have different attributes, beliefs, and behavioral rules. This diversity is a primary driver of complex dynamics.
2.  **Bounded Rationality:** Agents are not hyper-rational optimizers with perfect foresight. They often follow simple heuristics or rules of thumb (e.g., "imitate your successful neighbors"), which is arguably a more realistic description of human behavior. This concept traces its roots to the work of Herbert Simon.
3.  **Local Interaction:** Agents interact directly with a limited set of other agents or their local environment. There is no central "auctioneer" coordinating global equilibrium. This idea has intellectual origins in the work on **cellular automata** by mathematicians like John von Neumann and Stanisław Ulam in the 1940s.
4.  **Emergence:** Macroeconomic patterns (e.g., business cycles, market crashes, social norms) are not imposed on the model but are **emergent properties** that arise from the repeated, local interactions of the heterogeneous agents. The whole is more than the sum of its parts. Modern ABM research was significantly boosted by the interdisciplinary work at institutions like the Santa Fe Institute.

![Emergence Diagram](../images/png/emergence_diagram.png)

<a id='odd'></a>
### The ODD Protocol for Describing ABMs

Due to their complexity, documenting ABMs in a standardized way is crucial for reproducibility. The **ODD (Overview, Design Concepts, and Details) protocol** is the standard framework for this purpose. It provides a structured format for describing the model's purpose, components, and processes.

**1. Overview**
- **Purpose:** What is the model intended to explain or explore?
- **Entities, State Variables, and Scales:** What are the agents and other entities in the model? What are their attributes? What are the spatial and temporal scales?
- **Process Overview and Scheduling:** What are the basic actions agents perform, and in what order are they executed in each time step?

**2. Design Concepts**
- **Basic Principles:** What are the core theoretical principles underlying the model's design?
- **Emergence:** What key results are expected to emerge from the simulation?
- **Adaptation:** How do agents change their behavior over time?
- **Objectives:** What goals, if any, do the agents have?
- **Learning:** How do agents update their internal models or rules?
- **Sensing:** What can agents observe about their environment and other agents?
- **Interaction:** How do agents interact with each other?
- **Stochasticity:** Where is randomness used in the model?

**3. Details**
- **Initialization:** What is the initial state of the world? How are agent attributes and network connections set up?
- **Input Data:** Does the model use any external data?
- **Submodels:** What are the precise mathematical or logical formulations for the processes within the model (e.g., the exact rule for an agent's decision to move)?

Throughout this notebook, we will implicitly follow the ODD structure when describing each model.

<a id='schelling'></a>
## 2. Foundational Model 1: Schelling's Segregation Model

Thomas Schelling's 1971 model is a canonical ABM that provides a stunning demonstration of **emergence**. It shows that stark, macro-level patterns of residential segregation can arise even when all individuals have only a mild preference for not being in a local minority.

### Historical Context
Developed during the Civil Rights era, Schelling's model was a powerful tool for understanding how segregation could persist even without widespread, explicit racism. Schelling, a Nobel laureate in Economics, used a simple setup with coins on a chessboard to illustrate his theory, demonstrating that individual choices, even with mild preferences, could lead to dramatic and unintended social consequences.

**ODD Specification:**
- **Purpose:** To demonstrate how collective segregation patterns can emerge from weak individual preferences.
- **Entities:** Agents of different types (e.g., colors) and a grid representing a city, with some empty patches.
- **State Variables:** Each agent has a `type` and a `location`. Each patch has a `status` (empty or occupied by an agent of a certain type).
- **Process Overview:** In each step, all "unhappy" agents are identified. An agent is unhappy if the proportion of their neighbors of the same type is below their `tolerance` threshold. A random subset of unhappy agents is then moved to random empty locations.
- **Design Concepts:** The core concept is **emergence**. There is no central planner or explicit rule for segregation. The macro-pattern emerges from local, repeated interactions. Agents are **boundedly rational**, following a simple happiness rule rather than solving a global optimization problem.
- **Details:** The model is initialized with a random distribution of agents. The key parameter is the `tolerance` threshold.

In [ ]:
sec("Interactive Schelling Segregation Model")
class SchellingModel:
    """ Implements Schelling's segregation model. """
    def __init__(self, size=50, empty_ratio=0.1, tolerance=0.4, n_types=3):
        self.size, self.tolerance, self.n_types = size, tolerance, n_types
        self.segregation_history = []
        # The color map includes a color for the empty cells.
        self.cmap = ListedColormap(['#f0f0f0', 'crimson', 'royalblue', 'forestgreen', 'darkorange'][:n_types + 1])
        
        # The grid is initialized with agents and empty cells.
        n_cells = size**2
        n_agents = int(n_cells * (1 - empty_ratio))
        pop_list = np.array_split(np.arange(n_agents), n_types)
        pop_list = sum([[i+1]*len(arr) for i, arr in enumerate(pop_list)], [])
        population = np.array([0]*(n_cells - len(pop_list)) + pop_list)
        np.random.shuffle(population)
        self.grid = population.reshape((size, size))
        self._calculate_segregation()
        
    def _get_neighbors(self, r, c):
        """ Gets the neighbors of a cell, with wrap-around boundaries. """
        return [self.grid[(r+dr)%self.size, (c+dc)%self.size] for dr in [-1,0,1] for dc in [-1,0,1] if not (dr==0 and dc==0)]
    
    def _calculate_segregation(self):
        """ Calculates the average similarity of agents to their neighbors. """
        similarities = [np.mean([n == self.grid[r,c] for n in self._get_neighbors(r,c) if n!=0]) for r, c in np.argwhere(self.grid > 0) if any(n!=0 for n in self._get_neighbors(r,c))]
        self.segregation_history.append(np.mean(similarities) if similarities else 1.0)
        
    def step(self):
        """ Performs one step of the simulation. """
        # We identify agents who are not satisfied with their neighborhood.
        unhappy_agents = [loc for loc in np.argwhere(self.grid > 0) if np.mean([n == self.grid[tuple(loc)] for n in self._get_neighbors(*loc) if n != 0]) < self.tolerance]
        if not unhappy_agents: return 0
        
        # Unhappy agents are moved to random empty cells.
        empty_cells = list(zip(*np.where(self.grid == 0)))
        np.random.shuffle(unhappy_agents)
        np.random.shuffle(empty_cells)
        num_to_move = min(len(unhappy_agents), len(empty_cells))
        for i in range(num_to_move):
            self.grid[tuple(empty_cells[i])] = self.grid[tuple(unhappy_agents[i])]
            self.grid[tuple(unhappy_agents[i])] = 0
        self._calculate_segregation()
        return num_to_move

def run_and_plot_schelling(tolerance=0.4):
    """ Runs the Schelling model and plots the results. """
    model = SchellingModel(tolerance=tolerance, n_types=3, size=50)
    initial_grid = model.grid.copy()
    for i in range(150): 
        if model.step() == 0: break
    note(f"Tolerance {tolerance:.2f}: Converged after {i+1} iterations.")
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(initial_grid, cmap=model.cmap, interpolation='nearest')
    axes[0].set_title('Initial State')
    axes[1].imshow(model.grid, cmap=model.cmap, interpolation='nearest')
    axes[1].set_title('Final State')
    for ax in [axes[0], axes[1]]: 
        ax.set_xticks([])
        ax.set_yticks([])
    axes[2].plot(model.segregation_history, '-o', lw=2)
    axes[2].set_title('Segregation Index')
    axes[2].set(xlabel='Iteration', ylabel='Avg. Similarity', ylim=(0, 1))
    plt.tight_layout()
    plt.show()

widgets.interact(run_and_plot_schelling, tolerance=widgets.FloatSlider(min=0.1, max=0.8, step=0.05, value=0.4, description='Tolerance'));

<a id='schelling-analysis'></a>
### Emergence and Tipping Points
The interactive simulation reveals a crucial insight: there is a **tipping point**. Below a certain tolerance level (e.g., < 0.33), the society remains integrated. But once the tolerance crosses this critical threshold, a small change in individual preference leads to a dramatic, nonlinear shift in the collective outcome, resulting in a highly segregated equilibrium. This emergent result is a hallmark of complex systems and demonstrates how market-like mechanisms can fail to achieve socially optimal outcomes, as the final segregated state is often Pareto-dominated.

<a id='axelrod'></a>
## 3. Foundational Model 2: Axelrod's Tournament and the Evolution of Cooperation

Why does cooperation exist in a world of self-interested actors? Robert Axelrod's famous 1980s tournament provided a powerful answer using an agent-based approach. He invited experts to submit strategies (agents) for the **Iterated Prisoner's Dilemma (IPD)** game. In the IPD, two players repeatedly choose to either **Cooperate (C)** or **Defect (D)**. The payoffs are structured such that mutual defection is the single-round Nash Equilibrium, but mutual cooperation yields a higher collective payoff.

Axelrod's key finding was that the simplest strategy, **Tit-for-Tat** (start by cooperating, then copy your opponent's previous move), consistently won the tournament. It succeeded because it was **nice** (never the first to defect), **retaliatory** (punishing defection), **forgiving** (willing to cooperate again after being punished), and **clear** (its simple rule was easy for other strategies to recognize).

<a id='axelrod-impl'></a>
### Implementing the Iterated Prisoner's Dilemma
We can replicate Axelrod's findings by creating a population of agents with different strategies and simulating their evolution. Successful strategies (those that accumulate the highest payoffs) are more likely to "reproduce," meaning their proportion in the population grows over time.

In [ ]:
sec("Axelrod's Tournament Simulation")

# We define the strategies as functions. 
# Each function takes the agent's own history and the opponent's history as arguments.
def tit_for_tat(my_history, their_history): 
    """ Cooperates on the first move, then copies the opponent's last move. """
    return 'C' if not their_history else their_history[-1]
def always_defect(my_history, their_history): 
    """ Always defects. """
    return 'D'
def always_cooperate(my_history, their_history): 
    """ Always cooperates. """
    return 'C'
def grim_trigger(my_history, their_history): 
    """ Cooperates until the opponent defects, then defects forever. """
    return 'D' if 'D' in their_history else 'C'
def random_move(my_history, their_history): 
    """ Makes a random move. """
    return random.choice(['C', 'D'])

@dataclass
class AxelrodAgent:
    """ Represents an agent in the tournament. """
    strategy_func: callable
    name: str
    history: list = field(default_factory=list)
    score: float = 0.0

class AxelrodTournament:
    """ Simulates the tournament. """
    def __init__(self, population_counts):
        self.strategies = {'Tit-for-Tat': tit_for_tat, 'Always Defect': always_defect, 
                           'Always Cooperate': always_cooperate, 'Grim Trigger': grim_trigger, 'Random': random_move}
        self.population = []
        for name, count in population_counts.items():
            self.population.extend([AxelrodAgent(self.strategies[name], name) for _ in range(count)])
        self.payoffs = {('C', 'C'): (3, 3), ('C', 'D'): (0, 5), ('D', 'C'): (5, 0), ('D', 'D'): (1, 1)}

    def run_round_robin(self, n_rounds=10):
        """ Each agent plays against every other agent. """
        for i, p1 in enumerate(self.population):
            for j, p2 in enumerate(self.population[i+1:]):
                p1.history, p2.history = [], []
                for _ in range(n_rounds):
                    move1 = p1.strategy_func(p1.history, p2.history)
                    move2 = p2.strategy_func(p2.history, p1.history)
                    p1.history.append(move1); p2.history.append(move2)
                    score1, score2 = self.payoffs[(move1, move2)]
                    p1.score += score1; p2.score += score2

    def evolve(self):
        """ The population evolves based on the agents' scores. """
        total_score = sum(a.score for a in self.population)
        if total_score == 0: return # Avoids division by zero
        fitness = [a.score / total_score for a in self.population]
        new_population_agents = np.random.choice(self.population, size=len(self.population), p=fitness, replace=True)
        self.population = [AxelrodAgent(a.strategy_func, a.name) for a in new_population_agents]
        
    def run_simulation(self, n_generations=50):
        """ Runs the simulation for a number of generations. """
        history = []
        for _ in range(n_generations):
            counts = {name: 0 for name in self.strategies}
            for agent in self.population: counts[agent.name] += 1
            history.append(counts)
            self.run_round_robin()
            self.evolve()
        return pd.DataFrame(history)

# --- We set up and run the simulation ---
initial_counts = {'Tit-for-Tat': 25, 'Always Defect': 25, 'Always Cooperate': 25, 'Grim Trigger': 25}
tournament = AxelrodTournament(initial_counts)
history_df = tournament.run_simulation(n_generations=30)

fig, ax = plt.subplots(figsize=(14, 8))
history_df.plot(ax=ax, lw=3)
ax.set_title("Figure 2: Evolution of Strategies in Axelrod's Tournament", fontsize=18)
ax.set_xlabel('Generation')
ax.set_ylabel('Number of Agents')
ax.legend(title='Strategy')
ax.grid(True)
plt.show()

note("The simulation shows that 'nice' strategies like Tit-for-Tat and Grim Trigger tend to thrive, while 'Always Defect' can do well initially but ultimately falters as it eliminates its cooperative victims. 'Always Cooperate' is too naive and is quickly driven to extinction by defectors.")

<a id='finance-abm'></a>
## 4. Application 1: Financial Market ABMs

ABMs have become a powerful tool for studying financial markets, as they can generate many of the "stylized facts" of financial time series that are difficult to explain with standard representative-agent models. These facts include:
- **Fat Tails:** The distribution of returns has more extreme values than a normal distribution.
- **Volatility Clustering:** Periods of high volatility are followed by more high volatility, and vice versa.
- **Bubbles and Crashes:** Asset prices can deviate significantly from their fundamental values for extended periods.

<a id='kirman'></a>
### The Kirman (1993) "Ants" Model of Herd Behavior

This simple, elegant model shows how herd behavior can emerge from local interactions, even without any global information. Originally used to model the foraging behavior of ants, it's been widely adopted in finance. Agents can be in one of two states: **optimistic** (fundamentalist) or **pessimistic** (chartist). They change their state based on two factors:
1.  **Idiosyncratic Shocks:** An agent might randomly switch their opinion (with probability $\epsilon$).
2.  **Social Influence (Herding):** An agent meets another agent and may be convinced to adopt their opinion (with probability $h$).

Even though there is no underlying "fundamental" driving the state, the model can generate periods where one opinion becomes dominant purely through social contagion, mimicking the herd behavior seen in financial markets.

In [ ]:
sec("Kirman's Ant Model of Herd Behavior")

class KirmanAntsModel:
    """ Implements Kirman's ant model of herd behavior. """
    def __init__(self, N=100, epsilon=0.05, h=0.2):
        self.N, self.epsilon, self.h = N, epsilon, h
        self.n_optimists = N // 2 # We start with a 50/50 split of opinions.
        
    def step(self):
        """ Performs one step of the simulation. """
        # A random agent is chosen.
        agent_idx = random.randint(0, self.N - 1)
        is_optimist = agent_idx < self.n_optimists
        
        # The agent may switch opinion idiosyncratically.
        if random.random() < self.epsilon:
            if is_optimist: self.n_optimists -= 1
            else: self.n_optimists += 1
            return
        
        # Or the agent may be influenced by another agent.
        if random.random() < self.h:
            other_idx = random.randint(0, self.N - 1)
            other_is_optimist = other_idx < self.n_optimists
            if is_optimist and not other_is_optimist: self.n_optimists -= 1
            elif not is_optimist and other_is_optimist: self.n_optimists += 1
            
    def run(self, n_steps=5000):
        """ Runs the simulation for a number of steps. """
        history = np.zeros(n_steps)
        for t in range(n_steps):
            self.step()
            history[t] = self.n_optimists / self.N
        return history

# --- We set up and run the simulation ---
kirman_model = KirmanAntsModel(N=100, epsilon=0.01, h=0.3)
optimist_fraction = kirman_model.run(n_steps=10000)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), gridspec_kw={'height_ratios': [3, 1]})
fig.suptitle("Figure 3: Kirman's Model of Herding", fontsize=18)
ax1.plot(optimist_fraction, label='Fraction of Optimists')
ax1.set_title('Time Series of Agent States')
ax1.set_ylabel('Fraction')
ax1.set_ylim(0, 1)

ax2.hist(optimist_fraction, bins=50, density=True)
ax2.set_title('Distribution of States')
ax2.set_xlabel('Fraction of Optimists')
ax2.set_ylabel('Density')

for ax in [ax1, ax2]: ax.grid(True)
plt.tight_layout(rect=[0,0,1,0.96])
plt.show()

note("The model generates a bimodal distribution. The system spends most of its time in states where one opinion is dominant, with rapid switches between them. This occurs because the herding term ($h$) creates a positive feedback loop: the more optimists there are, the more likely a pessimist is to meet one and switch, and vice versa.")

<a id='asm'></a>
### An Artificial Stock Market (Lux & Marchesi)

Building on the idea of interacting agent types, we can construct a simple Artificial Stock Market (ASM). This model, in the spirit of Lux & Marchesi (1999), generates endogenous bubbles, crashes, and volatility clustering from the interaction of two types of traders:
1.  **Fundamentalists:** They believe the price will revert to a fundamental value $P_f$. Their demand for the stock is proportional to the perceived mispricing, $(P_f - P_t)$.
2.  **Chartists (or Noise Traders):** They extrapolate past trends. Their demand is proportional to the last observed price change, $(P_t - P_{t-1})$.

Agents can switch between these strategies based on which is performing better. The price is set by a **market maker** who adjusts the price in response to excess demand.
$$ P_{t+1} = P_t + \beta (D_t - S_t) $$
This feedback loop—where prices affect strategies, which affect demand, which in turn affects prices—is the engine of the model's complex dynamics.

In [ ]:
sec("Interactive Artificial Stock Market Simulation")

class ArtificialStockMarket:
    """ Implements a simple artificial stock market. """
    def __init__(self, n_agents=100, p_fundamental=100.0, beta=0.5, v=1.0):
        self.n_agents, self.p_fundamental, self.beta, self.v = n_agents, p_fundamental, beta, v
        self.prices = [p_fundamental, p_fundamental] # We start with two prices for trend calculation.
        self.n_chartists = n_agents // 2
        self.chartist_fraction_history = [0.5, 0.5]

    def step(self):
        """ Performs one step of the simulation. """
        n_fundamentalists = self.n_agents - self.n_chartists
        last_price, prev_price = self.prices[-1], self.prices[-2]
        
        # Agents determine their demand.
        demand_f = n_fundamentalists * (self.p_fundamental - last_price)
        demand_c = self.n_chartists * (last_price - prev_price)
        
        # The market maker adjusts the price based on excess demand.
        new_price = last_price + self.beta * (demand_f + demand_c) / self.n_agents
        self.prices.append(new_price)
        
        # Agents update their strategies based on performance.
        profit_f = (new_price - last_price) * (self.p_fundamental - last_price)
        profit_c = (new_price - last_price) * (last_price - prev_price)
        attraction = profit_c - profit_f
        frac_chartists = 1 / (1 + np.exp(-self.v * attraction))
        self.n_chartists = int(self.n_agents * frac_chartists)
        self.chartist_fraction_history.append(self.n_chartists / self.n_agents)

    def run(self, n_steps=500):
        """ Runs the simulation for a number of steps. """
        for _ in range(n_steps): self.step()
        return self.prices, self.chartist_fraction_history

def interactive_asm_plot(beta=0.3, v=0.1):
    """ Runs the artificial stock market and plots the results. """
    asm = ArtificialStockMarket(beta=beta, v=v)
    prices, chartist_fractions = asm.run(n_steps=500)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    fig.suptitle('Figure 4: An Artificial Stock Market', fontsize=18)
    ax1.plot(prices, label='Market Price', lw=2)
    ax1.axhline(asm.p_fundamental, color='k', linestyle='--', label='Fundamental Value')
    ax1.set_title('Endogenous Bubbles and Crashes')
    ax1.set_ylabel('Price')
    ax1.legend()
    ax1.grid(True)
    ax2.plot(chartist_fractions, 'r-', label='Fraction of Chartists')
    ax2.set_title('Evolution of Trader Population')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Fraction')
    ax2.set_ylim(0, 1)
    ax2.legend()
    ax2.grid(True)
    plt.tight_layout(rect=[0,0,1,0.96])
    plt.show()

note("Use the sliders to see how micro-level parameters affect macro-level outcomes. A higher `beta` (price sensitivity) or `v` (strategy switching speed) can lead to more volatile markets.")
widgets.interact(interactive_asm_plot, 
                 beta=widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.3, description='Price Speed (β)'),
                 v=widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description='Switch Speed (v)'));


<a id='macro-abm'></a>
## 5. Application 2: A Simple Agent-Based Macroeconomic Model

A growing body of research uses large-scale ABMs to analyze macroeconomic policy. These models, such as the one developed by Delli Gatti et al. (2011), feature thousands of interacting households, firms, and banks. They can generate endogenous business cycles, financial crises, and other complex dynamics from the interactions of these heterogeneous agents. Unlike DSGE models, they do not assume a single rational expectations equilibrium.

<a id='macro-impl'></a>
### Generating Endogenous Business Cycles
We can build a stylized model to demonstrate this. The model has two main agent types:
- **Households:** They consume a fraction of their income and save the rest. They supply labor to firms.
- **Firms:** They hire workers to produce a consumption good. They set prices based on a markup over their wage costs and adjust production based on demand. Firms that are unprofitable may go bankrupt, while successful firms can expand.

The interaction of these simple, boundedly rational rules can generate complex aggregate dynamics, including self-sustaining business cycles.

In [ ]:
sec("A Simple Agent-Based Macroeconomic Model")

@dataclass
class Household:
    """ Represents a household in the model. """
    id: int
    wealth: float = 10.0
    employed: bool = False

@dataclass
class Firm:
    """ Represents a firm in the model. """
    id: int
    capital: float = 100.0
    employees: list = field(default_factory=list)
    price: float = 1.0
    production_target: float = 10.0

class MacroABM:
    """ Implements a simple agent-based macroeconomic model. """
    def __init__(self, n_households=500, n_firms=50, consumption_propensity=0.8, markup=0.2):
        self.households = [Household(id=i) for i in range(n_households)]
        self.firms = [Firm(id=i) for i in range(n_firms)]
        self.unemployment_rate = 1.0
        self.gdp = 0
        self.params = {'c': consumption_propensity, 'm': markup}
        self.history = {'gdp': [], 'unemployment': []}

    def step(self):
        """ Performs one step of the simulation. """
        # Firms set their prices and production targets.
        avg_wage = 1.0 # This is a simplification for this model.
        for firm in self.firms:
            firm.price = (1 + self.params['m']) * avg_wage
            firm.production_target = 0.9 * firm.production_target + 0.1 * self.gdp / len(self.firms)
        
        # The labor market clears.
        for h in self.households: h.employed = False
        for f in self.firms: f.employees = []
        unemployed = list(self.households)
        random.shuffle(unemployed)
        for firm in self.firms:
            n_hires = int(firm.production_target / 10) # A simple production function.
            for _ in range(n_hires):
                if not unemployed: break
                worker = unemployed.pop()
                worker.employed = True
                firm.employees.append(worker)
        self.unemployment_rate = len(unemployed) / len(self.households)
        
        # Production and income are determined.
        self.gdp = 0
        for firm in self.firms:
            production = len(firm.employees) * 10
            self.gdp += production * firm.price
            for worker in firm.employees: worker.wealth += avg_wage * firm.price
            
        # Households consume.
        for household in self.households:
            if household.wealth > 0:
                consumption_amount = self.params['c'] * household.wealth
                household.wealth -= consumption_amount
        
        self.history['gdp'].append(self.gdp)
        self.history['unemployment'].append(self.unemployment_rate)

    def run(self, n_steps=200):
        """ Runs the simulation for a number of steps. """
        for _ in range(n_steps): self.step()
        return self.history

# --- We set up and run the simulation ---
macro_model = MacroABM()
history = macro_model.run()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
fig.suptitle('Figure 5: An Agent-Based Macroeconomic Model', fontsize=18)

ax1.plot(history['gdp'], label='GDP')
ax1.set_title('Endogenous Business Cycles')
ax1.set_ylabel('Nominal GDP')
ax1.grid(True)

ax2.plot(history['unemployment'], 'r-', label='Unemployment Rate')
ax2.set_title('Labor Market Dynamics')
ax2.set_xlabel('Time')
ax2.set_ylabel('Rate')
ax2.set_ylim(0, 1)
ax2.grid(True)
plt.show()

note("Even this highly stylized model generates persistent fluctuations in GDP and unemployment. The interaction between firms' production decisions and households' income and consumption creates feedback loops that drive the system, producing business-cycle-like dynamics without any external aggregate shocks.")

<a id='network-abm'></a>
## 6. Application 3: Opinion Dynamics on a Network

A key advantage of ABM is the ability to explicitly model interaction structures using networks. Let's demonstrate this with a simple **Voter Model**. Agents (nodes) on a network hold one of two opinions (e.g., 0 or 1). In each step, a random agent is chosen and they adopt the opinion of a randomly chosen neighbor. This simple local rule can lead to global consensus, but the time it takes depends critically on the network structure.

We will set up a **small-world network** (using the Watts-Strogatz model), which is known to have properties often seen in real social networks: high clustering and short average path lengths.

![Voter Model Animation](../images/10-Specialized-Models/voter_model_animation.gif)

<a id='validation'></a>
## 7. Strengths, Weaknesses, and Validation

**Strengths:**
- **Generality:** Can model complex systems with heterogeneity and non-linear interactions that are intractable with analytical methods.
- **Emergence:** Provides a framework for understanding how macro-phenomena arise from micro-level behavior.
- **Flexibility:** Easy to incorporate diverse agent behaviors, network structures, and institutional rules.

**Weaknesses:**
- **"Atheoretical":** A common criticism is that ABMs can become complex collections of ad-hoc rules without strong theoretical discipline.
- **Calibration and Validation:** With many parameters, it's difficult to rigorously calibrate the model to data. Validating that the model's success isn't just due to overfitting is a major challenge (the "curse of dimensionality").
- **Computational Cost:** Large-scale ABMs can be computationally intensive to simulate.

Validation often proceeds via **stylized fact replication**: does the model, for a plausible range of parameters, generate qualitative and quantitative patterns that match those seen in the real world (e.g., fat-tailed returns, volatility clustering)?

<a id='summary'></a>
## 7. Summary and Key Takeaways

This notebook provided a survey of the agent-based modeling approach, a powerful alternative to traditional equilibrium models.
- ABMs are a **bottom-up, generative** method for studying complex systems, built on the principles of **heterogeneity, bounded rationality, and local interaction**.
- Foundational models like **Schelling's segregation model** and **Axelrod's tournament** demonstrate how complex macro-patterns (segregation, cooperation) can **emerge** from simple micro-rules.
- **Financial ABMs**, like the Kirman and Lux-Marchesi models, can replicate key stylized facts of financial markets, such as bubbles, crashes, and fat-tailed returns, by modeling the interaction of agents with different strategies.
- **Macroeconomic ABMs** can generate endogenous business cycles and other aggregate dynamics from the interactions of households and firms, providing insights into out-of-equilibrium phenomena.
- While powerful, ABMs face significant challenges in **calibration and validation**, which remains an active area of research.

<a id='exercises'></a>
## 8. Exercises

1.  **Schelling's Tipping Point:** Modify the `run_and_plot_schelling` function to run the simulation for a range of `tolerance` values (e.g., from 0.2 to 0.6). Plot the final segregation index as a function of the tolerance level. What does the shape of this curve tell you about the system's non-linearity?

2.  **Breaking Cooperation:** In the `AxelrodTournament`, what happens if you add a 'Tit-for-Two-Tats' strategy (which only defects if the opponent defects twice in a row)? How does it fare against the original Tit-for-Tat? What happens if you introduce a small number of 'Always Defect' agents into a population dominated by Tit-for-Tat?

3.  **Kirman Model Dynamics:** In the `KirmanAntsModel`, how do the dynamics change as you vary the `epsilon` (idiosyncratic switching) and `h` (herding) parameters? What happens if `h` is zero? What happens if `epsilon` is very large? Relate these changes to the concepts of private information vs. social influence.

4.  **Artificial Stock Market Stability:** In the `ArtificialStockMarket` model, what is the effect of the strategy switching speed parameter, `v`? Run simulations with a very low `v` and a very high `v`. How does this parameter affect the size and frequency of bubbles and crashes?

5.  **Macro ABM Policy Experiment:** Add a government agent to the `MacroABM`. In a recession (e.g., when the unemployment rate exceeds a certain threshold), have the government increase spending (e.g., by hiring some of the unemployed workers directly). Does this policy stabilize the economy? How does it affect the government's debt over time?